# Задача 3.2
Дана статистика бросков для игроков NBA

Выборка представляет собой статистику бросков для различных игроков NBA. Для каждого игрока известны:

- количество успешных бросков в домашних играх (score_home)
- количество бросков в домашних играх (atm_home)
- количество успешных бросков в гостевых играх (score_away)
- количество бросков в гостевых играх (atm_away)

Требуется определить, есть ли разница в успехе бросков у игроков в домашних и гостевых играх.

У какого процента игроков разница в успехе существенна?

# Решение

## 1) Считываем данные

In [7]:
import pandas as pd
import numpy as np

In [22]:
data = pd.read_csv("data/nba.csv")
all_home = data.atm_home.values
suc_home = data.score_home.values
all_away = data.atm_away.values
suc_away = data.score_away.values
data.head()

,Unnamed: 0,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,7,A. Johnson,ATL,30,34,24,28,64,52
1,10,A. Johnson,TOR,63,57,44,38,120,82
2,15,Abdur-Rahim,SAC,114,124,84,90,238,174
3,22,Afflalo,DEN,236,284,189,227,520,416
4,23,Afflalo,DET,75,88,62,67,163,129


In [23]:
sum(all_home == 0), sum(suc_home == 0), sum(all_away == 0), sum(suc_away == 0)

(0, 0, 0, 0)

## 2) Описание метода

Для каждого игрока имеем по 2 выборки бинарных переменных. Чтобы сравнить эффективности попадания воспользуемся стандартным $Z$-критерием для разности двух долей (независимые выборки). Таким образом, для каждого игрока получим p-value. После поправки (Холма) на множественное тестирование найдем игроков, у которых частота попаданий в домашних и гостевых играх отличается статистически значимо.

In [25]:
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests

In [59]:
p_values = np.array([proportions_ztest([suc_home[i], suc_away[i]], [all_home[i], all_away[i]])[1] 
          for i in range(len(all_home))])

In [69]:
sum(p_values < 0.05), len(p_values), sum(p_values < 0.05)/len(p_values)

(65, 936, 0.06944444444444445)

In [61]:
rejected, p_values_corrected, _, __ = multipletests(p_values, method="holm")

In [62]:
sum(rejected)

0

Ни одного не отверглось, попробуем более мягкие поправки

In [63]:
rejected, p_values_corrected, _, __ = multipletests(p_values, method="fdr_bh")

In [64]:
sum(rejected)

0

In [65]:
rejected, p_values_corrected, _, __ = multipletests(p_values, method="fdr_by")

In [66]:
sum(rejected)

0

# Вывод
Таким образом не нашлось ни одного игрока, у которого бы частоты попаданий в домашних и гостевых играх отличались бы статистически значимо.